In [1]:
%matplotlib widget

In [2]:
from genSTEM import Model
import matplotlib.pyplot as plt
import numpy as np
import cupy as cp
from importlib import reload
reload(Model)
from tqdm.auto import tqdm

In [3]:
from ase.io import read
from ase.build import make_supercell

atoms = read("0013687130_v6bxv2_tv0.1bxv0.0_d1.8z_traj.xyz")
atoms = make_supercell(atoms, np.diag((1,2,1)))
mask = atoms.positions[:,2] > 35
del atoms[mask]
atoms.rotate([1,0,0], (0,0,1))

from skimage.transform import warp
from matplotlib.transforms import Affine2D

atoms[1716].number = 60
atoms[1766].number = 60

In [69]:
reload(Model)
0.0002

0.0002

In [114]:
%%time
ImageModel = Model.ImageModel

nImages = 4
pixel_size = 0.1
drift_strength = 10 # approx 10 -> becomes 10 / np.prod(image_shape)
jitter = 0.3
#drift_vector = [1,1]
random_angle = np.random.random() * 2*np.pi
drift_vector = [np.cos(random_angle),np.sin(random_angle)]

centre_drift = True

images = []

m0 = ImageModel(atoms, pixel_size=pixel_size, vacuum=10.,
                scan_rotation=0, centre_drift=centre_drift)
img0 = m0.generate_cupy()
side = np.minimum(*img0.shape) # Use to make image square

scanangles = np.linspace(0, 360, nImages, endpoint=False)
for scanangle in tqdm(scanangles):
    m = ImageModel(atoms, scan_rotation=scanangle,
                   pixel_size=pixel_size, vacuum=10.,
                    drift_strength=drift_strength, 
                    drift_vector=drift_vector,
                    jitter_strength=jitter, 
                    centre_drift=centre_drift)
    img = m.generate_cupy()

    img = img[:side, :side]
    images.append(img)


Wall time: 3.78 s


In [115]:
fig, AX = plt.subplots(6, 6, figsize=(10,10))
for ax, img in zip(AX.flatten(), images):
    ax.imshow(img.get())
    ax.axis('off')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [116]:
reload(Model)

<module 'genSTEM.Model' from 'c:\\users\\thomasaar\\documents\\github\\generate_scan_distorted_stem\\genstem\\genSTEM\\Model.py'>

In [117]:
best_angle, best_str = Model.estimate_drift(images, scanangles, tolerancy_percent=0.1)
print(f"\nActual angle: {np.round(np.rad2deg(random_angle), 2)}°")
print(f"Missed by {np.round(best_angle - np.rad2deg(random_angle),2)}°")

Iteration #0. Best guess: Angle: 0°, Strength: 0.00e+00


Iteration #1. Best guess: Angle: 270°, Strength: 3.60e-04


Iteration #2. Best guess: Angle: 292.5°, Strength: 3.00e-04


Iteration #3. Best guess: Angle: 290.0°, Strength: 3.20e-04


Iteration #4. Best guess: Angle: 289.17°, Strength: 3.20e-04


Iteration #5. Best guess: Angle: 289.44°, Strength: 3.20e-04


Iteration #6. Best guess: Angle: 289.54°, Strength: 3.20e-04

Took 4.2 seconds

Actual angle: 289.71°
Missed by -0.17°


In [118]:
mean, nonan = Model.create_final_stack_and_average(images, scanangles, best_str, best_angle)

In [119]:
plt.figure()
plt.imshow(mean)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [59]:
fig, AX = plt.subplots(2, 2, figsize=(10,10))
for ax, img in zip(AX.flatten(), nonan):
    ax.imshow(img)
    ax.axis('off')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [118]:
mean_image = cp.nanmean(warped_images2, axis=0)
mean_image[cp.isnan(mean_image)] = 0
#print(shifts)
plt.figure()
plt.imshow(mean_image.get())


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [57]:
fig, AX = plt.subplots(6, 6, figsize=(10,10))
for ax, img in zip(AX.flatten(), warped_images_nonan):
    ax.imshow(img.get())
    ax.axis('off')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [186]:
angle, strength = pairs[i]
i1 = Model.warp_image_cuda(img1, scanangle1, strength, angle)
i2 = Model.warp_image_cuda(img2, scanangle2, strength, angle)
#i3 = Model.warp_image_cuda(img3, scanangle3, strength, angle)

In [187]:
shift = shifts[i][0]
def translate(img, shift):
    return np.roll(img, shift, axis=(0,1))

A = i1.get()
B = translate(i2.get(), (shift[0], shift[1]))
#C = translate(i3.get(), (shift[0], shift[1]))


fig, (ax1, ax2, ax3, ) = plt.subplots(ncols=3, figsize=(9,5))
ax1.imshow(A)
ax2.imshow(B)
#ax3.imshow(C)
ax3.imshow(np.mean(np.asarray([A,B]), axis=0))


TypeError: 'int' object is not subscriptable

In [148]:
A,B,C = (img.get() for img in images[:3])
fig, (ax1, ax2, ax3, ax4) = plt.subplots(ncols=4, figsize=(12,10))
ax1.imshow(A)
ax2.imshow(B)
ax3.imshow(C)
ax4.imshow(np.mean(np.asarray([A,B,C]), axis=0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …